MultiTaskLasso 是 Lasso 回归在多任务学习场景下的扩展，用于同时处理多个相关的回归任务，并强制这些任务共享相同的特征选择模式。它在金融、医疗、气象预测等多目标预测场景中尤为有用。

### 1.基本原理

<B>多任务学习的核心思想</B>

    传统机器学习模式通常独立处理每个任务，但在实际应用中，多个任务之间可能存在相关性（例如，不同城市的天气预测、同一患者的多种疾病预测）。多任务学习通过共享特征表示，能同时提升所有任务的性能。

<B>MultiTaskLasso的目标函数<B>

给定 T 个相关的回归任务，每个任务 t 有 \(n_t\) 个样本和 p 个特征，MultiTaskLasso 同时优化所有任务的损失函数，并施加 组稀疏性约束：

以下是使用 sklearn 实现 MultiTaskLasso 的简单示例：

In [2]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import MultiTaskLasso
from sklearn.model_selection import train_test_split

In [27]:
# 生成多任务数据（前10个特征对所有任务有用，其余特征随机）
n_samples,n_features,n_tasks = 100, 20, 5

X, Y = make_regression(
    n_samples= n_samples,
    n_features=n_features,
    n_informative=10,
    n_targets=n_tasks,
    noise=20,
    random_state=49
)



In [28]:
X.shape
Y.shape

(100, 5)

In [29]:
# 划分训练集 和 测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [30]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(80, 20)
(20, 20)
(80, 5)
(20, 5)


In [31]:
# 训练 MultiTaskLasso 模型
mt_lasso = MultiTaskLasso(alpha=0.1, random_state=42)
mt_lasso.fit(X_train, Y_train)

MultiTaskLasso(alpha=0.1, random_state=42)

In [32]:
# 查看系数矩阵（形状：[n_features, n_tasks]）
print("系数矩阵形状:", mt_lasso.coef_.shape)

系数矩阵形状: (5, 20)


In [33]:
mt_lasso.coef_

array([[ 2.85289513e+01, -1.56058293e+00, -1.24264233e+00,
         9.94156361e+01,  3.01889573e+01,  4.45947720e+00,
        -2.76792454e+00,  1.16140134e+01,  2.05546019e+00,
         6.24626142e+01, -3.20696556e-01, -1.65361455e+00,
         5.10716673e+01,  3.59808861e+01,  2.14685448e+01,
         2.97190789e-01,  2.88106894e+00,  2.73382487e+00,
         2.46346481e+01,  6.04053147e+01],
       [ 5.74142404e+01,  8.04663255e-02, -2.38767570e+00,
         7.40168058e+01,  6.15621907e+01, -1.76173899e+00,
        -7.50635009e+00,  4.89425754e+01,  3.14204865e+00,
         1.85386144e+01,  6.27528929e+00,  2.18165344e+00,
         4.69267646e+01,  8.34711050e+01,  5.50681866e+01,
        -5.46534967e+00,  4.95300670e+00, -4.61591274e+00,
         1.02661092e+01,  7.86703732e+01],
       [ 9.55025632e+01,  3.17031599e-01,  1.71559948e+00,
         1.69844025e+01,  9.77461741e+01,  8.36564079e-01,
        -2.28050971e+00,  8.27284327e+01,  1.81911320e+00,
         5.01129537e+01,  2.4

In [37]:
# 验证组稀疏性：非零系数的特征是否对所有任务都非零
non_zero_features = np.sum(np.abs(mt_lasso.coef_) > 1e-10, axis=1)
non_zero_features == n_tasks

array([False, False, False, False, False])

In [35]:

print("对所有任务都有非零系数的特征数量:", np.sum(non_zero_features == n_tasks))
print("对任何任务都没有贡献的特征数量:", np.sum(non_zero_features == 0))

对所有任务都有非零系数的特征数量: 0
对任何任务都没有贡献的特征数量: 0
